In [31]:
import numpy as np
import operator
#np.set_printoptions(threshold=np.inf)

world = -1. * np.ones((18,14))
starts = [(17,3), (17,4), (17,5), (17,6)]
goals = [(1,13), (2,13), (3,13), (4,13), (5,13)]

beg_ = [(6,11),(7,9),(8,8),(9,8),(10,8),(11,8),(12,8),(13,7),(14,7),(15,7),(16,7),(17,7)]
end_ = [(0,13),(1,4),(2,0),(3,0),(4,0),(5,0),(6,1),(7,1),(8,1),(9,2),(10,2),(11,3),(12,3),(13,3),(14,3),(15,2),(16,2),(17,2)]

for coup in end_:
    world[coup[0], 0:coup[1]+1] = -10
    
for coup in beg_:
    world[coup[0], coup[1]:14] = -10

for cell in goals:
    world[cell] = -1

print(world)

[[-10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10. -10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.]
 [-10. -10.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1.  -1. -10. -10. -10.]
 [-10. -10.  -1.  -1.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10.]
 [-10. -10.  -1.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10.  -1.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10.  -1.  -1.  -1.  -1. -10. -10. -10. -10. -10. -10.]
 [-10. -10. -10. -10.  -1.  -1.  -1. -10. -10. -10.

In [109]:
MIN = 1
MAX = 5

EPS = 0.1

def generate_actions():
    actions = []
    for i in range(MAX+1):
        for j in range(MAX+1):
            if MIN <= (i + j) <= MAX:
                actions.append((i,j))
    return actions

def is_alive(state):
    # Out of bounds
    if ((state['y'] < 0 or state['y'] >= world.shape[0]) or 
        (state['x'] < 0 or state['x'] >= world.shape[1])):
        return False
    
    # Goal state
    elif ((state['y'], state['x']) in goals):
        return False
    
    # Obstacle hit
    elif world[state['y'], state['x']] == -10:
        return False
    
    else:
        return True

def move(state, action):
    new_state = state.copy()
    
    # One of the velocity is 0
    if action[0] == 0 or action[1] == 0:        
        # Index of non-zero velocity
        idx = 0 if action[1] == 0 else 1
        
        # Dictionary entry based on idx
        k = 'x' if action[1] == 0 else 'y'
        
        # Simulate trajectory
        for x in range(action[idx]):
            if k == "x":
                new_state[k] += 1
            else:
                new_state[k] -= 1
            
            # Check if alive
            if not is_alive(new_state):
                return new_state
            
    # Diagonal movement
    else:
        if action[0] >= action[1]:
            # Simulate trajectory
            for x in range(action[0]):
                new_state['y'] -= 1
                new_state['x'] += 1

                # Check if alive
                if not is_alive(new_state):
                    return new_state
        else:
            # Simulate trajectory
            for x in range(action[0]):
                new_state['y'] -= 1
                new_state['x'] += 1

                # Check if alive
                if not is_alive(new_state):
                    return new_state
            
            for x in range(action[1] - action[0]):
                new_state['y'] -= 1
                
                # Check if alive
                if not is_alive(new_state):
                    return new_state
    
    return new_state

def encode(state, action, R, Q, Pi):
    # Enconding key
    key_s = ','.join(str(x) for x in state.values())
    key_a = ','.join(str(x) for x in action)

    # Check if key not already in dict
    if key_s not in R.keys():
        R[key_s] = {}
    
    if key_s not in Q.keys():
        Q[key_s] = {}
        
    if key_s not in Pi.keys():
        Pi[key_s] = {}
        
    if key_a not in R[key_s].keys():
        R[key_s][key_a] = []
        
    return key_s, key_a

def monte_carlo(loops=1000, uniform=False):
    # Action-Value table
    R = {}
    Q = {}
    Pi = {}
    
    # Possible actions
    actions = generate_actions()
    
    for count in range(loops):
        # Start episode 
        alive = True

        # Starting state
        state = {'y': 17, 'x': np.random.randint(3,7)}
        # Episode states
        episode = []
        
        ## (a)
        while alive:
            # Action choice
            if uniform:
                action = actions[np.random.randint(0, len(actions))]
            else: 
                key_s = ','.join(str(x) for x in state.values())
                
                if key_s not in Pi.keys():
                    action = actions[np.random.randint(0, len(actions))]
                else:
                    a_star = max(Q[key_s].items(), key=operator.itemgetter(1))[0]
                    a_list = a_star.split(',')
                    action = (int(a_list[0]), int(a_list[1]))
                
            # Apply move to action
            new_state = move(state, action)
            
            # Encode state and action
            key_s, key_a = encode(state, action, R, Q, Pi)
            
            episode.append((key_s, key_a))
            
            # Check if new state valid
            if is_alive(new_state):
                # Update state
                state = new_state
            else:
                alive = False
        
        # (b)
        for key_s, key_a in episode:
            s_list = key_s.split(',')
            state = (int(s_list[0]), int(s_list[1]))

            # Update return dict
            R[key_s][key_a].append(world[state[0], state[1]])
            Q[key_s][key_a] = np.mean(R[key_s][key_a])
            
        # (c)
        if not uniform: 
            for key_s, _ in episode:
                a_star = max(Q[key_s].items(), key=operator.itemgetter(1))[0]
                a_len = len(actions)
                for action in actions:
                    key_a = ','.join(str(x) for x in action)
                    Pi[key_s][key_a] = (1 - EPS + EPS / a_len) if a_star == key_a else EPS / a_len
        
    return Pi

In [114]:
Pi = monte_carlo(loops=1000, uniform=False)

In [111]:

traj = []

for state_ in starts:
    # Episode states
    episode = []
    state = {'y': state_[0], 'x': state_[1]}
    while True:
        key_s = ','.join(str(x) for x in [state['y'],state['x']])

        a_star = max(Pi[key_s].items(), key=operator.itemgetter(1))[0]
        a_list = a_star.split(',')
        action = (int(a_list[0]), int(a_list[1]))
        
        print(Pi[key_s])
        print(action)
        
        # Apply move to action
        new_state = move(state, action)

        # Encode state and action
        key_s, key_a = encode(state, action, {}, {}, Pi)

        episode.append((key_s, key_a))

        # Check if new state valid
        if is_alive(new_state):
            # Update state
            state = new_state
        else:
            break
    traj.append(episode)


{'0,1': 0.005, '0,2': 0.005, '0,3': 0.005, '0,4': 0.005, '0,5': 0.905, '1,0': 0.005, '1,1': 0.005, '1,2': 0.005, '1,3': 0.005, '1,4': 0.005, '2,0': 0.005, '2,1': 0.005, '2,2': 0.005, '2,3': 0.005, '3,0': 0.005, '3,1': 0.005, '3,2': 0.005, '4,0': 0.005, '4,1': 0.005, '5,0': 0.005}
(0, 5)
{'0,1': 0.005, '0,2': 0.005, '0,3': 0.005, '0,4': 0.005, '0,5': 0.005, '1,0': 0.005, '1,1': 0.005, '1,2': 0.005, '1,3': 0.005, '1,4': 0.005, '2,0': 0.005, '2,1': 0.005, '2,2': 0.005, '2,3': 0.905, '3,0': 0.005, '3,1': 0.005, '3,2': 0.005, '4,0': 0.005, '4,1': 0.005, '5,0': 0.005}
(2, 3)
{'0,1': 0.005, '0,2': 0.005, '0,3': 0.005, '0,4': 0.005, '0,5': 0.005, '1,0': 0.005, '1,1': 0.005, '1,2': 0.005, '1,3': 0.005, '1,4': 0.005, '2,0': 0.005, '2,1': 0.005, '2,2': 0.005, '2,3': 0.005, '3,0': 0.005, '3,1': 0.905, '3,2': 0.005, '4,0': 0.005, '4,1': 0.005, '5,0': 0.005}
(3, 1)
{'0,1': 0.005, '0,2': 0.005, '0,3': 0.005, '0,4': 0.005, '0,5': 0.005, '1,0': 0.005, '1,1': 0.005, '1,2': 0.005, '1,3': 0.005, '1,4': 0.

In [112]:
traj

[[('17,3', '0,5')],
 [('17,4', '2,3'), ('14,6', '3,1')],
 [('17,5', '4,1')],
 [('17,6', '1,3')]]

In [115]:
Pi

{'17,6': {'0,1': 0.005,
  '0,2': 0.905,
  '0,3': 0.005,
  '0,4': 0.005,
  '0,5': 0.005,
  '1,0': 0.005,
  '1,1': 0.005,
  '1,2': 0.005,
  '1,3': 0.005,
  '1,4': 0.005,
  '2,0': 0.005,
  '2,1': 0.005,
  '2,2': 0.005,
  '2,3': 0.005,
  '3,0': 0.005,
  '3,1': 0.005,
  '3,2': 0.005,
  '4,0': 0.005,
  '4,1': 0.005,
  '5,0': 0.005},
 '15,6': {'0,1': 0.005,
  '0,2': 0.005,
  '0,3': 0.005,
  '0,4': 0.005,
  '0,5': 0.905,
  '1,0': 0.005,
  '1,1': 0.005,
  '1,2': 0.005,
  '1,3': 0.005,
  '1,4': 0.005,
  '2,0': 0.005,
  '2,1': 0.005,
  '2,2': 0.005,
  '2,3': 0.005,
  '3,0': 0.005,
  '3,1': 0.005,
  '3,2': 0.005,
  '4,0': 0.005,
  '4,1': 0.005,
  '5,0': 0.005},
 '10,6': {'0,1': 0.005,
  '0,2': 0.005,
  '0,3': 0.005,
  '0,4': 0.005,
  '0,5': 0.005,
  '1,0': 0.005,
  '1,1': 0.005,
  '1,2': 0.005,
  '1,3': 0.005,
  '1,4': 0.005,
  '2,0': 0.005,
  '2,1': 0.005,
  '2,2': 0.005,
  '2,3': 0.005,
  '3,0': 0.005,
  '3,1': 0.905,
  '3,2': 0.005,
  '4,0': 0.005,
  '4,1': 0.005,
  '5,0': 0.005},
 '17,4': {'0,